In [12]:
import pandas as pd
import numpy as np
import os
pd.options.display.max_columns = None

In [2]:
df=pd.read_csv("../../flight_data_raw.csv")

In [13]:
df.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,CANCELLATION_CODE,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,V26
0,2018,4,26,4,UA,DTW,EWR,910,1107.0,117.0,1.0,1053,1253.0,120.0,1.0,0,NaN,0,73.0,488,33.0,0.0,3.0,0.0,84.0,NaN
1,2018,4,26,4,UA,EWR,RDU,1400,1349.0,-11.0,0.0,1541,1521.0,-20.0,0.0,0,NaN,0,74.0,416,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,4,26,4,UA,SFO,EUG,1059,1115.0,16.0,1.0,1232,1242.0,10.0,0.0,0,NaN,0,61.0,451,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,4,26,4,UA,TPA,IAD,1858,1853.0,-5.0,0.0,2111,2058.0,-13.0,0.0,0,NaN,0,108.0,811,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,4,26,4,UA,SFO,BOS,1630,1625.0,-5.0,0.0,118,114.0,-4.0,0.0,0,NaN,0,316.0,2704,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
count=df.groupby(["ORIGIN","DEST"]).size()
count


ORIGIN  DEST
ABE     ATL      969
        CLT      955
        DTW      924
        FLL       73
        MDT        1
                ... 
YAK     CDV      361
        JNU      361
YNG     PIE        1
        SFB        1
YUM     PHX     1376
Length: 6365, dtype: int64

In [6]:
#this works to get airport list
#df2 = df[["ORIGIN","DEST"]].groupby('ORIGIN').aggregate(lambda tdf: tdf.unique().tolist())

In [7]:
len(df2.loc["ATL"]["DEST"])

NameError: name 'df2' is not defined

Atlanta has 166 Airports they fly to!

Create the final output csv:

In [99]:
x = df.groupby(["ORIGIN","DEST"]).size()

idx = pd.MultiIndex.from_product((df.ORIGIN.unique(), df.DEST.unique()))

r = x.reindex(idx).dropna()
r=r.reset_index()
          


In [125]:

y = df.groupby(["ORIGIN","DEST"]).mean()["ARR_DELAY"]

idx = pd.MultiIndex.from_product((df.ORIGIN.unique(), df.DEST.unique()))

r_2 = y.reindex(idx).dropna()

r_2=r_2.reset_index() 
#r_2.groupby('level_0').apply(lambda x: dict(zip(x['level_1'],x["ARR_DELAY"])))\

In [122]:
r_join= r.merge(r_2, how="left", on=["level_0","level_1"])
r_join.rename(columns= {0: "FLIGHT_VOLUME"}, inplace=True)

In [123]:

target=r_join.groupby('level_0')[['level_1',"FLIGHT_VOLUME","ARR_DELAY"]].apply(lambda x: x.set_index('level_1').to_dict(orient='index'))
target

level_0
ABE    {'FLL': {'FLIGHT_VOLUME': 73.0, 'ARR_DELAY': 3...
ABI    {'DFW': {'FLIGHT_VOLUME': 2022.0, 'ARR_DELAY':...
ABQ    {'SAT': {'FLIGHT_VOLUME': 19.0, 'ARR_DELAY': 3...
ABR    {'MSP': {'FLIGHT_VOLUME': 745.0, 'ARR_DELAY': ...
ABY    {'ATL': {'FLIGHT_VOLUME': 1018.0, 'ARR_DELAY':...
                             ...                        
WYS    {'SLC': {'FLIGHT_VOLUME': 239.0, 'ARR_DELAY': ...
XNA    {'EWR': {'FLIGHT_VOLUME': 325.0, 'ARR_DELAY': ...
YAK    {'CDV': {'FLIGHT_VOLUME': 361.0, 'ARR_DELAY': ...
YNG    {'SFB': {'FLIGHT_VOLUME': 1.0, 'ARR_DELAY': 9....
YUM    {'PHX': {'FLIGHT_VOLUME': 1376.0, 'ARR_DELAY':...
Length: 358, dtype: object

In [67]:
r

DTW  EWR    3873.0
     RDU    1315.0
     IAD    1863.0
     BOS    3734.0
     SAT     853.0
             ...  
EAR  DEN     200.0
ART  PHL      25.0
IFP  PHX      45.0
YNG  SFB       1.0
     PIE       1.0
Length: 6365, dtype: float64

In [127]:
target.to_csv("airports_serviced_data.csv")